# ABCDE Segmentation of spend

This notebook performs a simple ABCDE-segmentation of spend data, and exports results to .xlsx files.
Run each cell in the order they appear.

Scope (class2/brand) can be adjusted by changing the query. If changed, check compability for export function (exported columns).

### Imports, data loading & preprocessing

In [ ]:
import os, sys, re, numpy as np, pandas as pd
from pathlib import Path
from dotenv import load_dotenv

# Make attached helper modules importable (these are present next to the notebook or in /mnt/data)
EXTRA_MODULE_DIRS = [
    ".", "/mnt/data",  # adjust if your helpers live elsewhere
]
for p in EXTRA_MODULE_DIRS:
    if p not in sys.path:
        sys.path.insert(0, p)

# Import our helper utilities (from your attachments)
from source.data_prep import field_desc_utils as fdesc
from source.data_prep import field_value_utils as fval
from source.data_processing import analysis_utils as au
from source.data_processing import export_utils as xpu
import sql_queries as qreg
from source.db_connect import bigquery_connector

load_dotenv()  # reads .env in the working directory
PROJECT_ID = os.getenv("PROJECT_ID")
DATASET_ID = os.getenv("DATASET_ID")
TABLE_ID   = os.getenv("TABLE_ID")
OUTPUT_DIR = os.getenv("OUTPUT_DIR", "./output")
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
print("PROJECT_ID:", PROJECT_ID, "| DATASET_ID:", DATASET_ID, "| TABLE_ID:", TABLE_ID)


In [ ]:
# Build fully qualified table and fetch data
fqtn = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"
print("Reading from:", fqtn)

# Use correct import for BigQueryConnector
from source.db_connect import bigquery_connector
bq = bigquery_connector.BigQueryConnector(project_id=PROJECT_ID)  # uses default creds or GOOGLE_APPLICATION_CREDENTIALS
# Use analysis_utils registry to fetch the table you described
df = au.fetch_purchase_data_enriched(bq_client=bq)
print(df.shape, "rows x cols")
df.head(3)


In [ ]:
# Print columns for debugging after preprocessing
# Ensure df is defined before preprocessing!
try:
    df
except NameError:
    # If df is not defined, load it first
    fqtn = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"
    print("Reading from:", fqtn)
    bq = bigquery_connector.BigQueryConnector(project_id=PROJECT_ID)
    df = au.fetch_purchase_data_enriched(bq_client=bq)
    print(df.shape, "rows x cols")
    df.head(3)

df = au.preprocess_detailed_data(df)
print("Columns in DataFrame after preprocessing:", list(df.columns))

# Ensure key columns exist with consistent names
def _resolve(df, candidates):
    # mirror analysis_utils._resolve_col
    def norm(s): return re.sub(r"[^a-z0-9]", "", s.lower())
    by_norm = {norm(c): c for c in df.columns}
    for c in candidates:
        if c in df.columns: return c
        if norm(c) in by_norm: return by_norm[norm(c)]
    return candidates[0]  # fallback to first candidate, but user must check resolved value

# Force correct column names for robustness
COL_CLASS3 = "Class3"
COL_PNUM   = _resolve(df, ["ProductNumber"])
COL_PDESC  = _resolve(df, ["ProductDescription"])
COL_EUR    = _resolve(df, ["Amount Eur", "Purchase Amount Eur", "amount_eur", "purchase_amount_eur"])
COL_QTY    = _resolve(df, ["Purchase Quantity", "Quantity"])
print("Resolved:", {"Class3":COL_CLASS3, "ProductNumber":COL_PNUM, "ProductDescription":COL_PDESC, "€":COL_EUR, "Qty":COL_QTY})

# Normalize spend to numeric
df[COL_EUR] = pd.to_numeric(df[COL_EUR], errors="coerce").fillna(0.0)
df[COL_QTY] = pd.to_numeric(df[COL_QTY], errors="coerce")


### ABCDE Segmentation

In [ ]:
from source.data_processing.analysis_utils import compute_abcde_per_class3

df_seg = compute_abcde_per_class3(
    df,
    COL_CLASS3,   # class_col
    COL_PNUM,     # product_col
    COL_EUR,      # spend_col
    qty_col=COL_QTY,
    tiers=["A","B","C","D","E"],
    tier_thresholds=[0.8,0.95,0.99,0.999],
    min_products_per_tier=5,
    verbose=True
)

In [ ]:
# Run segmentation and merge results
tags = compute_abcde_per_class3(df, COL_CLASS3, COL_PNUM, COL_EUR)
df_seg = df.merge(tags, on=[COL_CLASS3, COL_PNUM], how="left")
df_seg.head(3)

In [ ]:
# Summarize tiers by Class3 (robust, no EUR mismatch, dedup tags)
# 1. Aggregate EUR per product in original df
# 2. Merge segmentation tags (deduplicated) onto this per-product EUR DataFrame
# 3. Use this for summary

# Aggregate EUR per product
per_product = df.groupby([COL_CLASS3, COL_PNUM], as_index=False)[COL_EUR].sum()
# Compute and deduplicate segmentation tags
seg_tags = compute_abcde_per_class3(df, COL_CLASS3, COL_PNUM, COL_EUR)
seg_tags = seg_tags.drop_duplicates(subset=[COL_CLASS3, COL_PNUM])
# Merge segmentation tags
per_product = per_product.merge(
    seg_tags,
    on=[COL_CLASS3, COL_PNUM], how="left"
)

# Print columns for debugging
print("Columns in per_product:", list(per_product.columns))

# Find EUR column by substring match (case-insensitive)
eur_cols = [col for col in per_product.columns if 'eur' in col.lower()]
print("Columns containing 'eur':", eur_cols)
if eur_cols:
    actual_eur_col = eur_cols[0]
    print(f"Using EUR column for aggregation: {actual_eur_col}")
else:
    raise KeyError(f"No column containing 'eur' found in per_product. Columns: {list(per_product.columns)}")

# Summary by Class3 and Segmentation
summary = (
    per_product.groupby([COL_CLASS3, "Segmentation"], as_index=False)
    .agg(
        products=(COL_PNUM, "nunique"),
        purchase_amount_eur_total=(actual_eur_col, "sum")
    )
    .sort_values([COL_CLASS3, "Segmentation"])
)
# Format # products as e.g. 1,000
summary["products_fmt"] = summary["products"].apply(lambda x: f"{x:,}")
# Format purchase_amount_eur_total as e.g. 1,000,000 EUR
summary["purchase_amount_eur_total_fmt"] = summary["purchase_amount_eur_total"].apply(lambda x: f"{x:,.0f} EUR")
summary[[COL_CLASS3, "Segmentation", "products_fmt", "purchase_amount_eur_total_fmt"]]

### Data validation step

In [ ]:
# Check that the sum of purchase_amount_eur_total in summary matches the raw total from the original table
total_summary = summary["purchase_amount_eur_total"].sum()
print(f"Summary tiers by Class3 total: {total_summary:,.0f} EUR")
if np.isclose(total_summary, raw_total):
    print("Summary tiers by Class3 total matches raw total from original table.")
else:
    print("Summary tiers by Class3 total does NOT match raw total from original table!")

In [ ]:
# Check the raw total from the original table (no deduplication)
raw_total = df[COL_EUR].sum()
print(f"Raw total from original table: {raw_total:,.0f} EUR")

### Export results to .xlsx

In [ ]:
# Print available columns in df_seg for debugging
print("Available columns in df_seg:", list(df_seg.columns))

# Try to select only columns that exist
export_cols = [
    COL_PNUM, COL_PDESC, COL_CLASS3, "Segmentation",
    "salesRounding_x", "year_authorization_x", COL_EUR + "_x",
    "head_shape_x","thread_type_x","head_height_x","head_outside_diameter_width_x","quality_x",
    "surface_treatment_x","material_x","din_standard_x","thread_diameter_x","length_x","height_x",
    "total_height_x","width_x","iso_standard_x","inside_diameter_x","outside_diameter_x",
    "thickness_x","designed_for_thread_x","total_length_x","head_type_x","thread_length_x"
]
existing_export_cols = [col for col in export_cols if col in df_seg.columns]

out_dir = Path(OUTPUT_DIR); out_dir.mkdir(exist_ok=True, parents=True)
written = xpu.export_year_split_purchase_quantity(
    bq,
    output_dir=str(out_dir),
    table=fqtn,                      # use enriched table
    fmt_thousands=True,
    merged_header_label="PurchaseQuantity",
    segmentation_df=df_seg[existing_export_cols].drop_duplicates(subset=[COL_PNUM], keep="first"),
    segmentation_col="Segmentation"
)
print("Files written:", len(written))
for p in written[:10]:
    print(" -", p)